In [30]:
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import gradio as gr

In [31]:
load_dotenv(override=True)

True

In [32]:
import os
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [33]:
class State(BaseModel):
    message: str

In [34]:
graph_builder = StateGraph(State)

In [35]:
model_name = "gemini-2.5-flash"
llm = ChatOpenAI(model=model_name)

In [36]:
def say_hello(old_state: State) -> State:

    response = llm.invoke(old_state["message"])
    return State(message=response.choices[0].message.content)

graph_builder.add_node("first_node", say_hello)

In [37]:
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

In [38]:
graph = graph_builder.compile()

In [39]:
def chat(user_input: str, history):
    state = State(message=user_input)
    result = graph.invoke(state)
    print(result)
    return result["message"]


gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [40]:

graph = graph_builder.compile()

Traceback (most recent call last):
  File "c:\work\workspace\ai-agents-playground\.venv\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\work\workspace\ai-agents-playground\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\work\workspace\ai-agents-playground\.venv\Lib\site-packages\gradio\blocks.py", line 2125, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\work\workspace\ai-agents-playground\.venv\Lib\site-packages\gradio\blocks.py", line 1605, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\work\workspace\ai-agents-playground\.venv\Lib\site-packages\gradio\utils.py", line 1033, in async_wrapper
   